# **Data analisys and prediction**

### Features:

| **Column** | **Description**                                           |
| ---------- | --------------------------------------------------------- |
| Survived   | person survived the titanic tragedy (0 - "NO", 1- "YES" ) |
| Sex        | the gender (female, male)                                 |
| Name       | the name                                                  |
| Age        | Age in years                                              |
| SibSp      | # of siblings/wife or husband on board                    |
| Parch      | # of parents / children on board                          |
| Pclass     | The ticket class (1 = "1st", 2 = "2nd", 3 = "3rd")        |

## Libraries to use:

In [1741]:
# %pip install pandas numpy seaborn pathlib matplotlib sklearn

In [1742]:
from pathlib import Path
import pandas as pd
import seaborn as sns
import numpy as np
import re
import matplotlib.pyplot as plt


In [1743]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.feature_selection import mutual_info_classif, SelectPercentile
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV


In [1744]:
file_path = Path("titanic.csv")
df_passengers = pd.read_csv(file_path)
df_passengers.head()

,Survived,Sex,Name,Age,SibSp,Parch,Pclass
0,1,male,"Anderson, Mr. Harry",48.0,0,0,1
1,1,female,"Andrews, Miss. Kornelia Theodosia",63.0,1,0,1
2,1,male,"Barkworth, Mr. Algernon Henry Wilson",80.0,0,0,1
3,0,male,"Baumann, Mr. John D",60.0,0,0,1
4,1,female,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",47.0,1,1,1


In [1745]:
df_passengers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  800 non-null    int64  
 1   Sex       800 non-null    object 
 2   Name      800 non-null    object 
 3   Age       800 non-null    float64
 4   SibSp     800 non-null    int64  
 5   Parch     800 non-null    int64  
 6   Pclass    800 non-null    int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 43.9+ KB


## Exploratory data analysis (EDA)

### Single variable analysis

In [1746]:
def show_pie_graph_of_a_variable(df_passengers, variable):
 grouping_by_variable=df_passengers.groupby([variable], observed=True)
 grouping_by_variable.size().plot.pie(y=variable,figsize=(5,5),autopct='%.2f%%')

In [1747]:
def show_bar_chart_of_a_variable(df_passengers,variable):
 grouping_by_variable=df_passengers.groupby([variable], observed=True)
 grouping_by_variable.size().plot.bar(y=variable,figsize=(5,5))